In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
import os, sys, gc, time, warnings, pickle, psutil, random
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
grid_df = pd.read_pickle('grid_df.pkl')

In [3]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20158410 entries, 0 to 20158409
Data columns (total 61 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         object 
 1   item_id                    int16  
 2   dept_id                    int8   
 3   store_id                   int8   
 4   cat_id                     int8   
 5   state_id                   int8   
 6   d                          int16  
 7   sales                      float16
 8   month                      int8   
 9   event_name_1               int8   
 10  event_type_1               int8   
 11  event_name_2               int8   
 12  event_type_2               int8   
 13  snap_CA                    float16
 14  snap_TX                    float16
 15  snap_WI                    float16
 16  sell_price                 float16
 17  release                    int16  
 18  day                        int8   
 19  week                       int8   
 20  

In [4]:
remove_feature = ['id',
                  'state_id',
                  'store_id',
                 'd','sales']

In [31]:
param_grid = {
                    'boosting_type': 'gbdt',
                    'objective': ['tweedie', 'poisson', 'regression']
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': [0.015, 0.01]
                    'num_leaves': [100,150,255]
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1000,
                    'boost_from_average': False,
                    'verbose': -1,
                    'seed' : 1995
                }

In [10]:
stores = list(grid_df['store_id'].unique())

In [5]:
validation = { 'public' : [1913, 1941], 'private' : [1941, 1969] }

In [6]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
tscv = TimeSeriesSplit(n_splits=3)

In [21]:
param_dist = {
    'boosting_type': ['gbdt', 'dart'],
    'objective': ['tweedie', 'poisson', 'regression'],
    'n_estimators': [500,1000],
    'metric': ['rmse'],
    'max_depth': [30, 50, 70],
    'num_leaves': [100, 255, 400],
    'learning_rate': [0.015, 0.02],
    'feature_fraction': [0.5, 0.7],
    'bagging_fraction': [0.5, 0.7],
}

In [22]:
df =  grid_df[grid_df['store_id'] == 1]

In [23]:
train = (df['d'] <= validation['public'][0]) & (df['d'] >= FIRST_DAY)
val = (df['d'] > validation['public'][0]) & (df['d'] <= validation['public'][1])

In [24]:
inputCol = grid_df.columns[~grid_df.columns.isin(remove_feature)]

In [25]:
X_train = df[train][inputCol]
Y_train = df[val]['sales']

X_test = df[train][inputCol]
Y_test = df[val]['sales']

In [26]:
reg = lgb.LGBMRegressor()

In [27]:
n_iter_search = 10
reg = lgb.LGBMRegressor()
random_search = RandomizedSearchCV(
    reg,
    param_distributions=param_dist,
    n_iter=n_iter_search,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1)

In [28]:
random_search.fit(
    X_train, 
    Y_train,
    eval_metric='rmse',
    eval_set=[(X_train, Y_train), (X_test, Y_test)],
    verbose=1,
)

[1]	training's rmse: 2.81856	valid_1's rmse: 3.16307
[2]	training's rmse: 2.80277	valid_1's rmse: 3.14595
[3]	training's rmse: 2.78706	valid_1's rmse: 3.12886
[4]	training's rmse: 2.7706	valid_1's rmse: 3.11047
[5]	training's rmse: 2.75392	valid_1's rmse: 3.09197
[6]	training's rmse: 2.73726	valid_1's rmse: 3.07315
[7]	training's rmse: 2.72041	valid_1's rmse: 3.05401
[8]	training's rmse: 2.70343	valid_1's rmse: 3.03479
[9]	training's rmse: 2.68627	valid_1's rmse: 3.01539
[10]	training's rmse: 2.66883	valid_1's rmse: 2.99583
[11]	training's rmse: 2.65128	valid_1's rmse: 2.97645
[12]	training's rmse: 2.63414	valid_1's rmse: 2.95703
[13]	training's rmse: 2.61759	valid_1's rmse: 2.93704
[14]	training's rmse: 2.60059	valid_1's rmse: 2.91713
[15]	training's rmse: 2.58352	valid_1's rmse: 2.89769
[16]	training's rmse: 2.56648	valid_1's rmse: 2.8789
[17]	training's rmse: 2.54981	valid_1's rmse: 2.85911
[18]	training's rmse: 2.5326	valid_1's rmse: 2.83984
[19]	training's rmse: 2.5162	valid_1's r

[154]	training's rmse: 1.83798	valid_1's rmse: 2.07458
[155]	training's rmse: 1.83704	valid_1's rmse: 2.07328
[156]	training's rmse: 1.83652	valid_1's rmse: 2.07257
[157]	training's rmse: 1.836	valid_1's rmse: 2.07201
[158]	training's rmse: 1.83532	valid_1's rmse: 2.07195
[159]	training's rmse: 1.83453	valid_1's rmse: 2.07389
[160]	training's rmse: 1.83406	valid_1's rmse: 2.07328
[161]	training's rmse: 1.83347	valid_1's rmse: 2.0727
[162]	training's rmse: 1.8329	valid_1's rmse: 2.07214
[163]	training's rmse: 1.83238	valid_1's rmse: 2.07156
[164]	training's rmse: 1.83147	valid_1's rmse: 2.0715
[165]	training's rmse: 1.83097	valid_1's rmse: 2.07112
[166]	training's rmse: 1.83037	valid_1's rmse: 2.07096
[167]	training's rmse: 1.83003	valid_1's rmse: 2.07056
[168]	training's rmse: 1.82956	valid_1's rmse: 2.07
[169]	training's rmse: 1.82913	valid_1's rmse: 2.06989
[170]	training's rmse: 1.8287	valid_1's rmse: 2.06956
[171]	training's rmse: 1.82839	valid_1's rmse: 2.06918
[172]	training's rm

[304]	training's rmse: 1.7792	valid_1's rmse: 2.05227
[305]	training's rmse: 1.77876	valid_1's rmse: 2.05222
[306]	training's rmse: 1.77842	valid_1's rmse: 2.05229
[307]	training's rmse: 1.77813	valid_1's rmse: 2.05229
[308]	training's rmse: 1.7778	valid_1's rmse: 2.05219
[309]	training's rmse: 1.77752	valid_1's rmse: 2.05216
[310]	training's rmse: 1.77718	valid_1's rmse: 2.05173
[311]	training's rmse: 1.77696	valid_1's rmse: 2.05135
[312]	training's rmse: 1.77677	valid_1's rmse: 2.05126
[313]	training's rmse: 1.7766	valid_1's rmse: 2.0513
[314]	training's rmse: 1.77638	valid_1's rmse: 2.05078
[315]	training's rmse: 1.77628	valid_1's rmse: 2.0507
[316]	training's rmse: 1.77602	valid_1's rmse: 2.05505
[317]	training's rmse: 1.77575	valid_1's rmse: 2.05509
[318]	training's rmse: 1.77556	valid_1's rmse: 2.05511
[319]	training's rmse: 1.77552	valid_1's rmse: 2.05509
[320]	training's rmse: 1.77537	valid_1's rmse: 2.05508
[321]	training's rmse: 1.77522	valid_1's rmse: 2.05507
[322]	training'

[455]	training's rmse: 1.74555	valid_1's rmse: 2.04779
[456]	training's rmse: 1.74552	valid_1's rmse: 2.0478
[457]	training's rmse: 1.7454	valid_1's rmse: 2.04787
[458]	training's rmse: 1.74501	valid_1's rmse: 2.04803
[459]	training's rmse: 1.74488	valid_1's rmse: 2.048
[460]	training's rmse: 1.7447	valid_1's rmse: 2.0477
[461]	training's rmse: 1.74455	valid_1's rmse: 2.04769
[462]	training's rmse: 1.74451	valid_1's rmse: 2.04762
[463]	training's rmse: 1.74441	valid_1's rmse: 2.04764
[464]	training's rmse: 1.74411	valid_1's rmse: 2.04763
[465]	training's rmse: 1.74378	valid_1's rmse: 2.04767
[466]	training's rmse: 1.74376	valid_1's rmse: 2.04767
[467]	training's rmse: 1.74352	valid_1's rmse: 2.04765
[468]	training's rmse: 1.74321	valid_1's rmse: 2.04768
[469]	training's rmse: 1.74316	valid_1's rmse: 2.04772
[470]	training's rmse: 1.74309	valid_1's rmse: 2.04798
[471]	training's rmse: 1.74297	valid_1's rmse: 2.04791
[472]	training's rmse: 1.74268	valid_1's rmse: 2.04789
[473]	training's

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=LGBMRegressor(), n_iter=5, n_jobs=-1,
                   param_distributions={'bagging_fraction': [0.5, 0.7],
                                        'boosting_type': ['gbdt'],
                                        'feature_fraction': [0.5, 0.7],
                                        'learning_rate': [0.015, 0.02],
                                        'max_depth': [30, 50, 70],
                                        'metric': ['rmse'],
                                        'n_estimators': [500, 1000],
                                        'num_leaves': [100, 255, 400],
                                        'objective': ['tweedie'],
                                        'tweedie_variance_power': [1.1]},
                   scoring='neg_root_mean_squared_error')

## random_search.best_params_